In [1]:
# Conditional Flow Matching on a 2-D Checkerboard
# - TRAIN: learn u_theta(x_t, t, c) ≈ (x1 - x0), where c = black-cell id
# - SAMPLE: Euler integrate x_{t+dt} = x_t + u_theta(x_t, t, c) * dt, with c drawn per particle
# - OUTPUT: a GIF showing particles flowing from N(0, I) into the checkerboard's black squares.

import math, os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter

import torch
import torch.nn as nn
import torch.nn.functional as F

# -----------------------
# Config
# -----------------------
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", DEVICE)

# Domain & data
BOARD_HALF_EXTENT = 4.0     # domain ~ [-L, L]^2
SQUARE_SIZE = 1.0
GAUSS_SIGMA = 1.2           # p0 = N(0, sigma^2 I)

# Training
BATCH_SIZE = 4096
STEPS = 10000               # increase to 15k–30k for crisper results
LR = 2e-3
PRINT_EVERY = 500
T_GAMMA = 2.0               # bias to early times: t = U() ** gamma (helps large displacements)

# Model size
HIDDEN = 256
X_FOURIER_M = 16            # number of Fourier pairs for x (2 -> 32-d)
T_EMB_DIM = 32              # sinusoidal time embedding dim
C_EMB_DIM = 32              # cell embedding dim

# Sampler / GIF
N_PARTICLES = 1000
N_STEPS = 240               # Euler steps (more => smoother)
FPS = 20
OUT_PATH = "checkerboard_condfm.gif"

# Reproducibility
RNG = np.random.default_rng(123)
torch.manual_seed(123)


Device: cuda


In [2]:
# -----------------------
# Checkerboard utilities
# -----------------------
def enumerate_black_cells(L=BOARD_HALF_EXTENT, s=SQUARE_SIZE):
    """Return list of black cell (i,j) covering [-L, L]^2 and a map (i,j)->id."""
    i_min = int(np.floor(-L / s)); i_max = int(np.ceil(L / s)) - 1
    j_min = int(np.floor(-L / s)); j_max = int(np.ceil(L / s)) - 1
    blacks = [(i, j) for i in range(i_min, i_max + 1)
                        for j in range(j_min, j_max + 1)
                        if ((i + j) % 2 == 0)]
    id_map = {ij: k for k, ij in enumerate(blacks)}
    return blacks, id_map

BLACKS, BLACK_ID = enumerate_black_cells()

def sample_checkerboard_given_cell_ids(cell_ids, L=BOARD_HALF_EXTENT, s=SQUARE_SIZE, rng=RNG):
    """Given integer cell_ids (0..num_black-1), sample uniform point inside each chosen black cell."""
    cells = np.array([BLACKS[int(cid)] for cid in cell_ids], dtype=np.int32)  # (N,2)
    u = rng.random(size=len(cell_ids))
    v = rng.random(size=len(cell_ids))
    x = (cells[:, 0] + u) * s
    y = (cells[:, 1] + v) * s
    return np.stack([x, y], axis=1).astype(np.float32)

def sample_checkerboard_uniform(n, rng=RNG):
    """Uniformly pick black cells, then uniform point inside each."""
    idx = rng.integers(0, len(BLACKS), size=n)
    return sample_checkerboard_given_cell_ids(idx, rng=rng), idx.astype(np.int64)

def sample_gaussian(n, sigma=GAUSS_SIGMA, rng=RNG):
    return rng.normal(0.0, sigma, size=(n, 2)).astype(np.float32)

def checker_image(L=BOARD_HALF_EXTENT, s=SQUARE_SIZE, res=400):
    x = np.linspace(-L, L, res)
    y = np.linspace(-L, L, res)
    X, Y = np.meshgrid(x, y)
    board = ((np.floor(X / s) + np.floor(Y / s)) % 2)
    return board

BOARD_IMG = checker_image()

def in_black_cells(pts):
    i = np.floor(pts[:,0] / SQUARE_SIZE).astype(int)
    j = np.floor(pts[:,1] / SQUARE_SIZE).astype(int)
    return ((i + j) % 2) == 0


In [3]:
# -----------------------
# Training batch sampler (Conditional FM)
# -----------------------
def sample_training_batch(batch_size=BATCH_SIZE):
    # Draw cell ids c, then x1 inside cell, x0 ~ N(0,I), t ~ U() ** gamma
    x1, c_ids = sample_checkerboard_uniform(batch_size, rng=RNG)
    x0 = sample_gaussian(batch_size, rng=RNG)
    u = RNG.random(size=(batch_size, 1)).astype(np.float32)
    t = u ** T_GAMMA
    xt = (1.0 - t) * x0 + t * x1              # supervision point
    v_star = x1 - x0                          # straight-transport velocity
    return (torch.from_numpy(xt),
            torch.from_numpy(t),
            torch.from_numpy(v_star),
            torch.from_numpy(c_ids))

# -----------------------
# Embeddings
# -----------------------
class FourierFeatures(nn.Module):
    def __init__(self, in_dim, m=16, scale=math.pi):
        super().__init__()
        B = torch.randn(in_dim, m) * scale     # random Fourier matrix
        self.register_buffer("B", B)           # (in_dim, m)
        self.out_dim = 2 * m

    def forward(self, x):                      # x: (B, in_dim)
        proj = x @ self.B                      # (B, m)
        return torch.cat([torch.sin(proj), torch.cos(proj)], dim=-1)

class TimeEmbed(nn.Module):
    def __init__(self, dim=32):
        super().__init__()
        # half sin, half cos with exponential frequency growth
        half = dim // 2
        freqs = 2.0 ** torch.linspace(0, 7, steps=half)
        self.register_buffer("freqs", freqs)
        self.out_dim = 2 * half

    def forward(self, t):                      # t: (B,1) in [0,1]
        ang = 2 * math.pi * t * self.freqs     # (B, half)
        return torch.cat([torch.sin(ang), torch.cos(ang)], dim=-1)

# -----------------------
# Vector field u_theta(x_t, t, c)
# -----------------------
class VectorField(nn.Module):
    def __init__(self, hidden=256, x_ff=X_FOURIER_M, t_dim=T_EMB_DIM, c_dim=C_EMB_DIM, num_cells=None):
        super().__init__()
        assert num_cells is not None
        self.xff = FourierFeatures(in_dim=2, m=x_ff, scale=math.pi)
        self.temb = TimeEmbed(dim=t_dim)
        self.cemb = nn.Embedding(num_cells, c_dim)
        in_dim = self.xff.out_dim + self.temb.out_dim + c_dim

        self.net = nn.Sequential(
            nn.Linear(in_dim, hidden), nn.SiLU(),
            nn.Linear(hidden, hidden), nn.SiLU(),
            nn.Linear(hidden, hidden), nn.SiLU(),
            nn.Linear(hidden, 2),
        )

        # Init
        for m in self.net:
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.zeros_(m.bias)

    def forward(self, x_t, t, c_ids):
        # Normalize coordinates to ~[-1,1] for stability
        x_scaled = x_t / BOARD_HALF_EXTENT
        h = torch.cat([self.xff(x_scaled), self.temb(t), self.cemb(c_ids)], dim=-1)
        return self.net(h)

NUM_CELLS = len(BLACKS)
u_theta = VectorField(hidden=HIDDEN, num_cells=NUM_CELLS).to(DEVICE)
opt = torch.optim.AdamW(u_theta.parameters(), lr=LR, weight_decay=1e-6)

# EMA for sampling stability
ema_decay = 0.999
u_ema = VectorField(hidden=HIDDEN, num_cells=NUM_CELLS).to(DEVICE)
u_ema.load_state_dict(u_theta.state_dict())

@torch.no_grad()
def ema_update():
    for p, q in zip(u_ema.parameters(), u_theta.parameters()):
        p.mul_(ema_decay).add_(q, alpha=1 - ema_decay)




In [4]:
# -----------------------
# Train (Conditional FM)
#   Minimize || u_theta(x_t, t, c) - (x1 - x0) ||^2
# -----------------------
u_theta.train()
for step in range(1, STEPS + 1):
    xt, t, v_star, c_ids = sample_training_batch(BATCH_SIZE)
    xt = xt.to(DEVICE); t = t.to(DEVICE); v_star = v_star.to(DEVICE); c_ids = c_ids.to(DEVICE)

    v_pred = u_theta(xt, t, c_ids)
    loss = F.mse_loss(v_pred, v_star)

    opt.zero_grad(set_to_none=True)
    loss.backward()
    torch.nn.utils.clip_grad_norm_(u_theta.parameters(), 1.0)
    opt.step()
    ema_update()

    if step % PRINT_EVERY == 0 or step == 1:
        print(f"[{step:5d}/{STEPS}] loss={loss.item():.6f}")

# -----------------------
# Sampling with Euler (Conditional)
# For each particle, draw a cell id c and integrate to t=1.
# -----------------------
@torch.no_grad()
def euler_sample(n_particles=N_PARTICLES, n_steps=N_STEPS, use_ema=True):
    model = u_ema if use_ema else u_theta
    model.eval()

    x = torch.from_numpy(sample_gaussian(n_particles, rng=RNG)).to(DEVICE)     # x0 ~ N(0,I)
    c_ids = torch.from_numpy(RNG.integers(0, NUM_CELLS, size=n_particles).astype(np.int64)).to(DEVICE)
    dt = 1.0 / n_steps
    traj = [x.detach().cpu().numpy()]
    for i in range(n_steps):
        t = torch.full((n_particles, 1), float(i) / n_steps, device=DEVICE)
        v = model(x, t, c_ids)
        x = x + v * dt
        traj.append(x.detach().cpu().numpy())
    return traj, c_ids.detach().cpu().numpy()

traj, c_ids_final = euler_sample()



[    1/10000] loss=6.698785
[  500/10000] loss=0.390675
[ 1000/10000] loss=0.369099
[ 1500/10000] loss=0.364557
[ 2000/10000] loss=0.367157
[ 2500/10000] loss=0.366888
[ 3000/10000] loss=0.373828
[ 3500/10000] loss=0.351827
[ 4000/10000] loss=0.357462
[ 4500/10000] loss=0.352630
[ 5000/10000] loss=0.387828
[ 5500/10000] loss=0.370972
[ 6000/10000] loss=0.354141
[ 6500/10000] loss=0.352950
[ 7000/10000] loss=0.338274
[ 7500/10000] loss=0.345776
[ 8000/10000] loss=0.346402
[ 8500/10000] loss=0.350825
[ 9000/10000] loss=0.351458
[ 9500/10000] loss=0.356989
[10000/10000] loss=0.362717


In [5]:
# -----------------------
# Make GIF
# -----------------------
fig, ax = plt.subplots(figsize=(6, 6))
L = BOARD_HALF_EXTENT
ax.set_xlim(-L, L); ax.set_ylim(-L, L); ax.set_aspect('equal')
ax.set_xticks([]); ax.set_yticks([])
#ax.imshow(BOARD_IMG, extent=[-L, L, -L, L], origin='lower', interpolation='nearest')
sc = ax.scatter(traj[0][:,0], traj[0][:,1], s=10)

def update(frame):
    sc.set_offsets(traj[frame])
    return (sc,)

anim = FuncAnimation(fig, update, frames=len(traj), blit=True)
writer = PillowWriter(fps=FPS)
anim.save(OUT_PATH, writer=writer, dpi=150)
plt.close(fig)

print(f"Saved GIF to: {OUT_PATH}")

# -----------------------
# Final sanity check
# -----------------------
final = traj[-1]
mask_black = in_black_cells(final)
print(f"in black cells: {mask_black.mean()*100:.2f}% (target ~100%)")

# (Optional) also check whether each final particle's position lies in its conditioned cell
def cell_id_from_xy(pts):
    ii = np.floor(pts[:,0] / SQUARE_SIZE).astype(int)
    jj = np.floor(pts[:,1] / SQUARE_SIZE).astype(int)
    ids = np.array([BLACK_ID.get((i, j), -1) for i, j in zip(ii, jj)], dtype=np.int64)
    return ids

ids_from_final_xy = cell_id_from_xy(final)
acc_match = (ids_from_final_xy == c_ids_final).mean() * 100.0
print(f"match conditioned cell id: {acc_match:.2f}% (higher is better)")

Saved GIF to: checkerboard_condfm.gif
in black cells: 91.10% (target ~100%)
match conditioned cell id: 90.40% (higher is better)
